In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
df = pd.read_excel('/content/drive/My Drive/train.xlsx')

In [3]:
df.head()

,Sl.no,Questions,Answers
0,1,What do you understand by Natural Language Pro...,Natural Language Processing is a field of comp...
1,2,What are stop words?,Stop words are said to be useless data for a s...
2,3,List any two real-life applications of Natural...,"Google Translate, Chatbots"
3,4,What is TF-IDF?,TFIDF or Term Frequency-Inverse Document Frequ...
4,5,What is Syntactic Analysis?,Syntactic Analysis is a technique of analyzing...


In [4]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer

def clean_sentence(sentence, stopwords=False):

  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)

  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(df,stopwords=False):
  sents=df[["Questions"]]
  cleaned_sentences=[]

  for index,row in df.iterrows():
    #print(index,row)
    cleaned=clean_sentence(row["Questions"],stopwords);
    cleaned_sentences.append(cleaned);
  return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['understand natural language processing', 'stop words', 'list reallife applications natural language processing', 'tfidf', 'syntactic analysis', 'semantic analysis', 'nltk', 'tokenize sentence nltk package', 'explain parsing', 'explain stemming', 'pragmatic analysis', 'pragmatic ambiguity', 'unigrams bigrams trigrams ngrams nlp', 'feature extraction nlp', 'partsofspeech tagging', 'explain named entity recognition', 'nlp process removing words like sentence called', 'tfidf helps establish', 'nlp process identifying people organization given sentence paragraph called', 'explain briefly word2vec']


['what do you understand by natural language processing', 'what are stop words', 'list any two reallife applications of natural language processing', 'what is tfidf', 'what is syntactic analysis', 'what is semantic analysis', 'what is nltk', 'how to tokenize a sentence using the nltk package', 'explain how we can do parsing', 'explain stemming', 'what is pragmatic analysis', 'what is pragmati

Bag of words Model

In [5]:
import numpy

sentences=cleaned_sentences_with_stopwords

#split it by white space
sentence_words = [[word for word in document.split() ]
                  for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key,' : ', value)
import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]

for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)
question_orig="What are the applications of NLP?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())

print("\n\n",question,"\n",question_embedding)

0  :  by
1  :  do
2  :  language
3  :  natural
4  :  processing
5  :  understand
6  :  what
7  :  you
8  :  are
9  :  stop
10  :  words
11  :  any
12  :  applications
13  :  list
14  :  of
15  :  reallife
16  :  two
17  :  is
18  :  tfidf
19  :  analysis
20  :  syntactic
21  :  semantic
22  :  nltk
23  :  a
24  :  how
25  :  package
26  :  sentence
27  :  the
28  :  to
29  :  tokenize
30  :  using
31  :  can
32  :  explain
33  :  parsing
34  :  we
35  :  stemming
36  :  pragmatic
37  :  ambiguity
38  :  and
39  :  bigrams
40  :  in
41  :  ngrams
42  :  nlp
43  :  trigrams
44  :  unigrams
45  :  extraction
46  :  feature
47  :  partsofspeech
48  :  tagging
49  :  entity
50  :  named
51  :  recognition
52  :  an
53  :  as
54  :  called
55  :  from
56  :  like
57  :  process
58  :  removing
59  :  establish
60  :  helps
61  :  given
62  :  identifying
63  :  organization
64  :  paragraph
65  :  people
66  :  about
67  :  briefly
68  :  word2vec
what do you understand by natural language p

In [6]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retriveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sin=-1;
    index_sin=-1;
    for index,faq_embeddings in enumerate(sentence_embeddings):
      #print(cosine_similarity(faq_embeddings,question_embedding))
      sin=cosine_similarity(faq_embeddings,question_embedding)[0][0];
      #print("SIN: ", sin)
      print(index, sin, sentences[index])
      if sin>max_sin:
          max_sin=sin;
          index_sin=index;
      

    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrived: ",FAQdf.iloc[index_sin,1])
    print(FAQdf.iloc[index_sin,2])

retriveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 0.1643989873053573 what do you understand by natural language processing
1 1.0 what are stop words
2 0.9557790087219501 list any two reallife applications of natural language processing
3 1.0 what is tfidf
4 1.0 what is syntactic analysis
5 1.0 what is semantic analysis
6 1.0 what is nltk
7 0.9928414716338373 how to tokenize a sentence using the nltk package
8 0.8137334712067349 explain how we can do parsing
9 0.9910476003087769 explain stemming
10 1.0 what is pragmatic analysis
11 1.0 what is pragmatic ambiguity
12 1.0 what are unigrams bigrams trigrams and ngrams in nlp
13 1.0 what is feature extraction in nlp
14 1.0 what is partsofspeech tagging
15 0.9910476003087769 explain named entity recognition
16 0.9978569490503114 in nlp the process of removing words like and is a an the from a sentence is called as
17 0.9997296931968458 tfidf helps you to establish
18 0.9956001436530537 in nlp the process of identifying people an organization from a given sentence paragraph is called
19 0.

In [7]:
from gensim.models import Word2Vec
import gensim.downloader as api

glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")

v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved w2v model")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);


[==================================================] 100.0% 104.8/104.8MB downloaded
Saved glove model
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved w2v model


In [8]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)

def getPhraseEmbedding(phrase,embeddingmodel):

        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
          den=den+1;
          vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        return vec.reshape(1, -1)

In [9]:
#with w2vec

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retriveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);


0 0.40878282752495915 understand natural language processing
1 0.26417283400975394 stop words
2 0.5524535805139236 list reallife applications natural language processing
3 0.0 tfidf
4 0.22370505728877457 syntactic analysis
5 0.2582587419587579 semantic analysis
6 0.0 nltk
7 0.1632663466133746 tokenize sentence nltk package
8 0.2744365775588105 explain parsing
9 0.15298253147396465 explain stemming
10 0.2540902553814592 pragmatic analysis
11 0.23255324518394263 pragmatic ambiguity
12 0.0 unigrams bigrams trigrams ngrams nlp
13 0.23620148839441313 feature extraction nlp
14 0.1410028698368078 partsofspeech tagging
15 0.21998634152933544 explain named entity recognition
16 0.34671536513131596 nlp process removing words like sentence called
17 0.19946497309473787 tfidf helps establish
18 0.3873620549346011 nlp process identifying people organization given sentence paragraph called
19 0.1949722949281446 explain briefly word2vec


Question:  what are the applications of nlp


Retrived:  List 